In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [23]:
#weekend_Jan_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]
weekend_Jan_list = [1, 8, 15, 22, 29]

In [24]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in weekend_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [25]:
len(link_day_minute_Jan_list)

14400

In [26]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [27]:
x[:,:2]

matrix([[ 352345.58212337,  349365.8698338 ],
        [ 362347.24968738,  358920.06835082],
        [ 656135.70404203,  659239.85596047],
        [ 666770.66182925,  673110.47886094],
        [ 175713.13948471,  176057.02508629],
        [ 235679.51835631,  233766.20179   ],
        [ 432851.94024934,  435375.80488904],
        [ 428862.53520429,  429748.31783243],
        [ 383197.77744651,  385983.94394234],
        [ 394997.56567081,  389055.95486397],
        [ 385868.75458212,  399346.76289193],
        [ 333535.8401955 ,  332337.34755611],
        [ 100046.15481047,   99823.31056737],
        [  99792.47154631,  103649.59555173],
        [ 166183.42761702,  155502.73524167],
        [ 148183.81282739,  147046.25125201],
        [ 221276.13104043,  216012.37518491],
        [ 226625.39293639,  223939.54326923],
        [ 371649.50288845,  376225.11111807],
        [ 333177.28884753,  344099.70331262],
        [ 499745.10215935,  516255.86797819],
        [ 531704.51494075,  524323

In [28]:
np.size(x, 0), np.size(x, 1)

(24, 482)

In [29]:
np.size(A,0), np.size(A,1)

(24, 140)

In [30]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.99102555e+07  0.00000000e+00  6.48e+03 1.84e+03  1.00e+06     0s
   1  -2.34510572e+07 -1.31034886e+07  1.19e+03 3.37e+02  2.43e+05     0s
   2  -1.61706169e+07 -1.82722124e+07  5.13e+01 1.46e+01  5.60e+04     0s
   3  -1.64143530e+07 -1.67323543e+07  5.11e-01 1.45e-01  5.8

In [31]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [32]:
lam_list

[3.5159416869315265e-08,
 0.00045500574563622775,
 182180.39436760606,
 0.0010935674213217172,
 0.0016263173646806628,
 242864.98611843798,
 145567.39242888804,
 5.0883781290834e-08,
 2.174840116410656e-09,
 264193.96455410204,
 2.1748401607096322e-09,
 2.1748401988210327e-09,
 2.1748429629017213e-09,
 2.1895029690336323e-09,
 375813.87621259247,
 1.7563045538405628e-09,
 1.4930780180754874e-06,
 314818.2311969843,
 165410.0483733567,
 0.00011831506666272473,
 0.02908496054276455,
 107451.06906722338,
 483780.863619679,
 9.448900787162318e-06,
 20825.99101235908,
 18437.382833307238,
 0.00014971230440069538,
 236348.1261563415,
 242731.63509678785,
 1.7563045653695282e-09,
 8.87717479013464e-06,
 2.221109917268163e-06,
 146573.35442489953,
 8.876139809003893e-07,
 1319.3761090890987,
 2.132592080195811e-06,
 1.7548593199861474e-09,
 1.700266500361371e-06,
 2.1414699499584973e-06,
 97101.08777617534,
 226458.4616362522,
 0.00314053612945564,
 419230.4169693175,
 1.7563045511264575e-09,
